In [1]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
with open('../Data/shakespeare.txt', 'r', encoding='utf8') as f:
    text = f.read()

In [5]:
type(text)

str

In [7]:
print(text[:100])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose mi


In [8]:
len(text)

5445609

In [9]:
all_characters = set(text)

In [10]:
all_characters

{'\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}'}

In [11]:
len(all_characters)

84

In [16]:
# num --> letter
decoder = dict(enumerate(all_characters))
decoder

{0: 'f',
 1: '(',
 2: 'B',
 3: 'n',
 4: '5',
 5: '<',
 6: '_',
 7: ')',
 8: 'H',
 9: '2',
 10: '}',
 11: 'v',
 12: '8',
 13: ']',
 14: 'F',
 15: 'V',
 16: ',',
 17: ';',
 18: 'X',
 19: 'T',
 20: 'q',
 21: '?',
 22: 'o',
 23: 'l',
 24: 'b',
 25: '|',
 26: '6',
 27: '[',
 28: '\n',
 29: '.',
 30: 'w',
 31: 'P',
 32: 'a',
 33: "'",
 34: 'S',
 35: 'p',
 36: 'G',
 37: '>',
 38: 't',
 39: 'E',
 40: 'k',
 41: 'W',
 42: '4',
 43: 'N',
 44: 'e',
 45: 'z',
 46: 'J',
 47: '"',
 48: 'g',
 49: 'i',
 50: '3',
 51: 'd',
 52: 'I',
 53: 'c',
 54: 'Y',
 55: 'U',
 56: 'D',
 57: '9',
 58: 'R',
 59: 'm',
 60: '`',
 61: 'O',
 62: 'u',
 63: '-',
 64: 'M',
 65: 'y',
 66: '!',
 67: 'Z',
 68: 'K',
 69: 'L',
 70: 'j',
 71: '1',
 72: 'h',
 73: '7',
 74: 'x',
 75: 's',
 76: '0',
 77: 'r',
 78: 'A',
 79: ':',
 80: 'C',
 81: '&',
 82: 'Q',
 83: ' '}

In [13]:
# letter --> num
encoder = {char: ind for ind, char in decoder.items()}

In [15]:
encoder

{'f': 0,
 '(': 1,
 'B': 2,
 'n': 3,
 '5': 4,
 '<': 5,
 '_': 6,
 ')': 7,
 'H': 8,
 '2': 9,
 '}': 10,
 'v': 11,
 '8': 12,
 ']': 13,
 'F': 14,
 'V': 15,
 ',': 16,
 ';': 17,
 'X': 18,
 'T': 19,
 'q': 20,
 '?': 21,
 'o': 22,
 'l': 23,
 'b': 24,
 '|': 25,
 '6': 26,
 '[': 27,
 '\n': 28,
 '.': 29,
 'w': 30,
 'P': 31,
 'a': 32,
 "'": 33,
 'S': 34,
 'p': 35,
 'G': 36,
 '>': 37,
 't': 38,
 'E': 39,
 'k': 40,
 'W': 41,
 '4': 42,
 'N': 43,
 'e': 44,
 'z': 45,
 'J': 46,
 '"': 47,
 'g': 48,
 'i': 49,
 '3': 50,
 'd': 51,
 'I': 52,
 'c': 53,
 'Y': 54,
 'U': 55,
 'D': 56,
 '9': 57,
 'R': 58,
 'm': 59,
 '`': 60,
 'O': 61,
 'u': 62,
 '-': 63,
 'M': 64,
 'y': 65,
 '!': 66,
 'Z': 67,
 'K': 68,
 'L': 69,
 'j': 70,
 '1': 71,
 'h': 72,
 '7': 73,
 'x': 74,
 's': 75,
 '0': 76,
 'r': 77,
 'A': 78,
 ':': 79,
 'C': 80,
 '&': 81,
 'Q': 82,
 ' ': 83}

In [17]:
encoded_text = np.array([encoder[char] for char in text])

In [19]:
encoded_text[:500]

array([28, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83,
       83, 83, 83, 83, 83, 71, 28, 83, 83, 14, 77, 22, 59, 83,  0, 32, 49,
       77, 44, 75, 38, 83, 53, 77, 44, 32, 38, 62, 77, 44, 75, 83, 30, 44,
       83, 51, 44, 75, 49, 77, 44, 83, 49,  3, 53, 77, 44, 32, 75, 44, 16,
       28, 83, 83, 19, 72, 32, 38, 83, 38, 72, 44, 77, 44, 24, 65, 83, 24,
       44, 32, 62, 38, 65, 33, 75, 83, 77, 22, 75, 44, 83, 59, 49, 48, 72,
       38, 83,  3, 44, 11, 44, 77, 83, 51, 49, 44, 16, 28, 83, 83,  2, 62,
       38, 83, 32, 75, 83, 38, 72, 44, 83, 77, 49, 35, 44, 77, 83, 75, 72,
       22, 62, 23, 51, 83, 24, 65, 83, 38, 49, 59, 44, 83, 51, 44, 53, 44,
       32, 75, 44, 16, 28, 83, 83,  8, 49, 75, 83, 38, 44,  3, 51, 44, 77,
       83, 72, 44, 49, 77, 83, 59, 49, 48, 72, 38, 83, 24, 44, 32, 77, 83,
       72, 49, 75, 83, 59, 44, 59, 22, 77, 65, 79, 28, 83, 83,  2, 62, 38,
       83, 38, 72, 22, 62, 83, 53, 22,  3, 38, 77, 32, 53, 38, 44, 51, 83,
       38, 22, 83, 38, 72

In [20]:
decoder[27]

'['

In [22]:
def one_hot_encoder(encoded_text, num_uni_chars):
    
    # encoded_text --> batch of encoded text
    # num_uni_chars --> len(set(text))
    
    one_hot = np.zeros((encoded_text.size, num_uni_chars))
    
    one_hot = one_hot.astype(np.float32)
    
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()]  = 1.0
    
    one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
    
    return one_hot

In [23]:
arr = np.array([1, 2, 0])
arr

array([1, 2, 0])

In [25]:
one_hot_encoder(arr, 3)

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)